In [1]:
%cd /content/drive/MyDrive/ImageBind finetune/ImageBind-LoRA

/content/drive/MyDrive/ImageBind finetune/ImageBind-LoRA


In [2]:
!pip install -r requirements.txt --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 97.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to buil

In [2]:
import os
current_directory = os.getcwd()

In [3]:
import torch
import logging
import data
from models import imagebind_model
from models.imagebind_model import ModalityType, load_module
from models import lora as LoRA

logging.basicConfig(level=logging.INFO, force=True)

lora = False
linear_probing = True
device = "cpu"  # "cuda:0" if torch.cuda.is_available() else "cpu"
load_head_post_proc_finetuned = True

assert not (linear_probing and lora), \
    "Linear probing is a subset of LoRA training procedure for ImageBind. " \
    "Cannot set both linear_probing=True and lora=True. "

if lora and not load_head_post_proc_finetuned:
    lora_factor = 12 / 0.07
else:
    lora_factor = 1


image_paths = ["test_data/00088_2.jpg", "test_data/00095.jpg", "test_data/00158_4.jpg",
               "test_data/00169_3.jpg", "test_data/00220.jpg", "test_data/00222_3.jpg"]
audio_paths = ["test_data/00088.wav", "test_data/00095.wav", "test_data/00158_4.wav",
               "test_data/00169_3.wav", "test_data/00220.wav", "test_data/00222_3.wav"]

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [4]:
for i in range(len(image_paths)):
    image_paths[i] = current_directory + "/" + image_paths[i]
for i in range(len(audio_paths)):
    audio_paths[i] = current_directory + "/" + audio_paths[i]

In [5]:
model = imagebind_model.imagebind_huge(pretrained=True)

In [6]:
if lora:
    model.modality_trunks.update(
        LoRA.apply_lora_modality_trunks(model.modality_trunks, rank=4,
                                        modality_names=[ModalityType.TEXT, ModalityType.VISION]))

    LoRA.load_lora_modality_trunks(model.modality_trunks,
                                   checkpoint_dir=".checkpoints/lora/", postfix="_last")

    if load_head_post_proc_finetuned:
        load_module(model.modality_postprocessors, module_name="postprocessors",
                    checkpoint_dir=".checkpoints/lora/", postfix="_last")
        load_module(model.modality_heads, module_name="heads",
                    checkpoint_dir=".checkpoints/lora/", postfix="_last")
elif linear_probing:
    load_module(model.modality_heads, module_name="heads",
                checkpoint_dir="./.checkpoints/lora/", postfix="_last")

model.eval()
model.to(device)

hello


INFO:root:Loaded parameters for module heads from ./.checkpoints/lora/.


ImageBindModel(
  (modality_preprocessors): ModuleDict(
    (vision): RGBDTPreprocessor(
      (cls_token): tensor((1, 1, 1280), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Sequential(
          (0): PadIm2Video()
          (1): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
        )
      )
      (pos_embedding_helper): SpatioTemporalPosEmbeddingHelper(
        (pos_embed): tensor((1, 257, 1280), requires_grad=True)
        
      )
    )
    (text): TextPreprocessor(
      (pos_embed): tensor((1, 77, 1024), requires_grad=True)
      (mask): tensor((77, 77), requires_grad=False)
      
      (token_embedding): Embedding(49408, 1024)
    )
    (audio): AudioPreprocessor(
      (cls_token): tensor((1, 1, 768), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10), bias=False)
        (norm_layer): LayerNorm((768,), eps=1e-05, elementwise_affine=

In [20]:
# Load data
audio_query_paths = [current_directory + "/test_data/00222_3.wav"]  # Example: Choose one audio query

inputs_audio_query = {
    ModalityType.AUDIO: data.load_and_transform_audio_data(audio_query_paths, device),
}

inputs_image = {
    ModalityType.VISION: data.load_and_transform_vision_data(image_paths, device),
}

# Generate embeddings for audio query
with torch.no_grad():
    embeddings_audio_query = model(inputs_audio_query)
with torch.no_grad():
    embeddings_image = model(inputs_image)

vision_embeddings = embeddings_image[ModalityType.VISION]
audio_query_embedding = embeddings_audio_query[ModalityType.AUDIO]


similarity_scores = torch.softmax(vision_embeddings @ audio_query_embedding.T * (lora_factor if lora else 1), dim=0)

most_relevant_image_index = torch.argmax(similarity_scores)

most_relevant_image_path = image_paths[most_relevant_image_index]

print("Most relevant image for audio query:", most_relevant_image_path)

Most relevant image for audio query: /content/drive/MyDrive/ImageBind finetune/ImageBind-LoRA/test_data/00220.jpg


In [21]:
vision_embeddings.shape

torch.Size([6, 1024])

In [22]:
vision_embeddings

tensor([[ 0.0324, -0.0125,  0.0028,  ..., -0.0556,  0.0091,  0.0193],
        [ 0.0296, -0.0076,  0.0037,  ..., -0.0237,  0.0210, -0.0054],
        [ 0.0415, -0.0056,  0.0056,  ..., -0.0377,  0.0092,  0.0189],
        [ 0.0101, -0.0157, -0.0177,  ..., -0.0316, -0.0348,  0.0001],
        [ 0.0086, -0.0168,  0.0137,  ..., -0.0459, -0.0009, -0.0122],
        [-0.0010, -0.0139, -0.0006,  ..., -0.0249, -0.0098, -0.0182]])

In [23]:
audio_query_embedding.T.shape

torch.Size([1024, 1])

In [24]:
torch.softmax(vision_embeddings @ audio_query_embedding.T,dim=0)

tensor([[0.0056],
        [0.1056],
        [0.0040],
        [0.0107],
        [0.5318],
        [0.3423]])

In [25]:
torch.softmax(vision_embeddings @ audio_query_embedding.T,dim=1)

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])

In [26]:
similarity_scores

tensor([[0.0056],
        [0.1056],
        [0.0040],
        [0.0107],
        [0.5318],
        [0.3423]])